## Installation

In [ ]:
%pip install -q pyomo
!apt-get install coinor-cbc

#### Initializing libraries and model

In [ ]:
from pyomo.environ import *
import math
model=ConcreteModel()
model.constraints=ConstraintList()

# Modelling

Add initial variables here, like importing the game files, the rules, etc, which are constants,or decided by the game

In [ ]:
n=15                                          #Number of match ticks
E=[840,840,840,2*840,2*840,2*840,4*840,2*840,2*840,840,2*840,840,840,840,1000]   #Enemy strength for each attack tick
E[n-1]=100000
T=2500                                        #Treasure required to reach victory
V=[70,56,56,56,56,70,84,252,42,84,140,84,56,98,1]             #Number of jobs in each match tick
Cg=8                                         #Amount of gold one can generate in each job for one job tick
Cf=8                                         #Amount of food one can generate in each job for one job tick
Ct=7                                         #Amount of treasure one can generate in each job for one job tick
Tg=30                                         #Amount of gold required to train one soldier
Tf=40                                         #Amount of food required to train one soldier
M=10000000                                    #The big M we will be using
Wininc=100000                                 #The army winning increase, needs to be optimized for better solution
Str_Sol=140                                    #Strength of one soldier
armyV_initial=140*24                           #Initialized as needed, the initial value of army
initial_gold=700
initial_food=1000

#### Instruction for modelling
Add variables by ``model.x=Var(range(_dimension_),domain=[_the domain of the variable_])``

In [ ]:
model.food_in=Var(range(n), domain=NonNegativeIntegers)                 #Number of jobs allocated to food in each tick
model.gold_in=Var(range(n),domain=NonNegativeIntegers)                  #Number of jobs allocated to gold in each tick
model.treasure_in=Var(range(n),domain=NonNegativeIntegers)              #Number of jobs allocated to treasure in each tick
model.total_gold=Var(range(n),domain=NonNegativeIntegers)               #Represents gold before ith attack {deducting all costs}
model.total_food=Var(range(n),domain=NonNegativeIntegers)               #Represents food before ith attack {deducting all costs}
model.total_treasure=Var(range(n),domain=NonNegativeIntegers)           #Represents treasure before ith attack {deducting all costs}
model.train=Var(range(n),domain=NonNegativeIntegers)                    #Represents the number of soldiers trained before ith attack
model.army=Var(range(n),domain=NonNegativeIntegers)                     #Represents the strength of total army before ith attack
model.armyV=Var(range(n),domain=NonNegativeIntegers)                    #Represents the strength of total army after ith attack
model.W=Var(range(n),domain=Binary)                                     #Represents binary which denotes whether won at or before ith attack or not
model.r=Var(range(n),domain=[1.0000,0.3333,0.6667,0.5000,0.2000,0.8000,0.0000,0.5714])  #Represents the  fraction of army which should survive
model.x1_25=Var(range(n),domain=Binary)                                 #The next few variables sets rule for wars, the damage our army gets in attack
model.x1_5=Var(range(n),domain=Binary)
model.x1_75=Var(range(n),domain=Binary)
model.x2=Var(range(n),domain=Binary)
model.x3=Var(range(n),domain=Binary)
model.x5=Var(range(n),domain=Binary)

Objective function

In [ ]:
model.obj = Objective(expr = n-sum(model.W[i] for i in range (n)), sense = minimize)          #Gives the fastest possible win, collecting T amounts of treasure

Add constraints by ``model.constraint.add(expr=...)``

In [ ]:
for i in range(1,n):
    model.constraints.add(expr=model.total_gold[i]==model.total_gold[i-1]+Cg*model.gold_in[i]-Tg*model.train[i])
    model.constraints.add(expr=model.total_food[i]==model.total_food[i-1]+Cf*model.food_in[i]-Tf*model.train[i])
    model.constraints.add(expr=model.total_treasure[i]==model.total_treasure[i-1]+Ct*model.treasure_in[i])
    model.constraints.add(expr=model.army[i]==model.armyV[i-1]+Str_Sol*model.train[i]+Wininc*model.W[i])
    model.constraints.add(expr=model.armyV[i]>=0)
    model.constraints.add(expr=model.army[i]>=E[i])
    model.constraints.add(expr=model.armyV[i]==model.army[i]-E[i]*model.r[i])
    #Now comes warzone constraints
    model.constraints.add(expr=model.army[i]-1.25*E[i]-1<=M*model.x1_25[i])           #x_{1.25}
    model.constraints.add(expr=model.army[i]-1.25*E[i]-1>=-M*(1-model.x1_25[i]))
    model.constraints.add(expr=model.r[i]-0.8000<=M*(1-model.x1_25[i]))
    model.constraints.add(expr=model.r[i]-0.8000>=-M*(model.x1_25[i]))

    model.constraints.add(expr=model.army[i]-1.5*E[i]-1<=M*model.x1_5[i])             #x_{1.5}
    model.constraints.add(expr=model.army[i]-1.5*E[i]-1>=-M*(1-model.x1_5[i]))
    model.constraints.add(expr=model.r[i]-0.6667<=M*(1-model.x1_5[i]))
    model.constraints.add(expr=model.r[i]-0.6667>=-M*(model.x1_5[i]))

    model.constraints.add(expr=model.army[i]-1.75*E[i]-1<=M*model.x1_75[i])           #x_{1.75}
    model.constraints.add(expr=model.army[i]-1.75*E[i]-1>=-M*(1-model.x1_75[i]))
    model.constraints.add(expr=model.r[i]-0.5714<=M*(1-model.x1_5[i]))
    model.constraints.add(expr=model.r[i]-0.5714>=-M*(model.x1_5[i]))

    model.constraints.add(expr=model.army[i]-2*E[i]-1<=M*model.x2[i])                 #x_{2}
    model.constraints.add(expr=model.army[i]-2*E[i]-1>=-M*(1-model.x2[i]))
    model.constraints.add(expr=model.r[i]-0.5000<=M*(1-model.x2[i]))
    model.constraints.add(expr=model.r[i]-0.5000>=-M*(model.x2[i]))

    model.constraints.add(expr=model.army[i]-3*E[i]-1<=M*model.x3[i])                 #x_{3}
    model.constraints.add(expr=model.army[i]-3*E[i]-1>=-M*(1-model.x3[i]))
    model.constraints.add(expr=model.r[i]-0.3333<=M*(1-model.x3[i]))
    model.constraints.add(expr=model.r[i]-0.3333>=-M*(model.x3[i]))

    model.constraints.add(expr=model.army[i]-5*E[i]-1<=M*model.x5[i])                 #x_{5}
    model.constraints.add(expr=model.army[i]-5*E[i]-1>=-M*(1-model.x5[i]))
    model.constraints.add(expr=model.r[i]-0.2000<=M*(1-model.x5[i]))
    model.constraints.add(expr=model.r[i]-0.2000>=-M*(model.x5[i]))

    model.constraints.add(expr=model.r[i]<=1-model.x5[i])

model.constraints.add(expr=model.total_treasure[0]==0)
model.constraints.add(expr=model.armyV[0]==armyV_initial)
model.constraints.add(expr=model.total_gold[0]==initial_gold)
model.constraints.add(expr=model.total_food[0]==initial_food)


for i in range(n):
    model.constraints.add(expr=model.total_gold[i]>=0)
    model.constraints.add(expr=model.total_food[i]>=0)
    model.constraints.add(expr=model.food_in[i]+model.gold_in[i]+model.treasure_in[i]==V[i])
    model.constraints.add(expr=model.total_treasure[i]<=M*model.W[i]+T-1)
    model.constraints.add(expr=model.total_treasure[i]>=-M*(1-model.W[i])+T-1)
    model.constraints.add(expr=model.r[i]<=1)
    model.constraints.add(expr=model.r[i]>=0)
    model.constraints.add(expr=model.W[i]>=0)
    model.constraints.add(expr=model.W[i]<=1)

## Solver using CBC

In [ ]:
opt = SolverFactory('cbc')
result=opt.solve(model,tee=True)
print('Solver status: ',result.solver.status)
print('Solver termination condition: ',result.solver.termination_condition)

# Resulting simulation

In [ ]:
for i in range(n):
  print(f"total gold at {i}: {value(model.total_gold[i])}")
print('\n')
for i in range(n):
  print(f"total food{i}: {value(model.total_food[i])}")
print('\n')
for i in range(n):
  print(f"total treasure{i}: {value(model.total_treasure[i])}")
print('\n')
for i in range(1,n):
  print(f"train{i}: {value(model.train[i])}")
print('\n')
for i in range(1,n):
  print(f"army{i}: {value(model.army[i])}")
print('\n')
for i in range(1,n):
  print(f"armyV{i}: {value(model.armyV[i])}")
print('\n')
print("Optimal N:", model.obj())

total gold at 0: 700.0
total gold at 1: 700.0
total gold at 2: 700.0
total gold at 3: 670.0
total gold at 4: 670.0
total gold at 5: 370.0
total gold at 6: 370.0
total gold at 7: 378.0
total gold at 8: 378.0
total gold at 9: 378.0
total gold at 10: 378.0
total gold at 11: 378.0
total gold at 12: 378.0
total gold at 13: 378.0
total gold at 14: 378.0


total food0: 1000.0
total food1: 1000.0
total food2: 1000.0
total food3: 960.0
total food4: 960.0
total food5: 560.0
total food6: 560.0
total food7: 560.0
total food8: 560.0
total food9: 568.0
total food10: 568.0
total food11: 568.0
total food12: 568.0
total food13: 1344.0
total food14: 1344.0


total treasure0: 0.0
total treasure1: 392.0
total treasure2: 784.0
total treasure3: 1176.0
total treasure4: 1568.0
total treasure5: 2058.0
total treasure6: 2646.0
total treasure7: 4403.0
total treasure8: 4697.0
total treasure9: 5278.0
total treasure10: 6258.0
total treasure11: 6846.0
total treasure12: 7238.0
total treasure13: 7245.0
total treasure14

# The decisions taken-

In [ ]:
print("Villager asssigned to resources-")
for i in range(0,n):
  print(f"Time {i}-")
  print(f"Food: {int(value(model.food_in[i]))}")
  print(f"Gold: {int(value(model.gold_in[i]))}")
  print(f"Treasure: {int(value(model.treasure_in[i]))}")
  print('\n')
print('\n')
print("Soldiers trained-")
for i in range(1,n):
  print(f"Time {i}: {int(value(model.train[i]))}")

Villager asssigned to resources-
Time 0-
Food: 0
Gold: 70
Treasure: 0


Time 1-
Food: 0
Gold: 0
Treasure: 56


Time 2-
Food: 0
Gold: 0
Treasure: 56


Time 3-
Food: 0
Gold: 0
Treasure: 56


Time 4-
Food: 0
Gold: 0
Treasure: 56


Time 5-
Food: 0
Gold: 0
Treasure: 70


Time 6-
Food: 0
Gold: 0
Treasure: 84


Time 7-
Food: 0
Gold: 1
Treasure: 251


Time 8-
Food: 0
Gold: 0
Treasure: 42


Time 9-
Food: 1
Gold: 0
Treasure: 83


Time 10-
Food: 0
Gold: 0
Treasure: 140


Time 11-
Food: 0
Gold: 0
Treasure: 84


Time 12-
Food: 0
Gold: 0
Treasure: 56


Time 13-
Food: 97
Gold: 0
Treasure: 1


Time 14-
Food: 0
Gold: 0
Treasure: 1




Soldiers trained-
Time 1: 0
Time 2: 0
Time 3: 1
Time 4: 0
Time 5: 10
Time 6: 0
Time 7: 0
Time 8: 0
Time 9: 0
Time 10: 0
Time 11: 0
Time 12: 0
Time 13: 0
Time 14: 0


## File to be piped to $game.o$

In [ ]:
file_name='best_strategy.txt'
with open(file_name,'w') as file:
  for i in range (1,n):
    file.write(str(int(model.food_in[i]()))+'\n')
    file.write(str(int(model.gold_in[i]()))+'\n')
    file.write(str(int(model.train[i]()))+'\n')